In [1]:
import pandas as pd
import networkx as nx
from operator import itemgetter
import matplotlib as m

In [2]:
# read the mondo hierarchy file
mondo_hie=pd.read_csv("/Users/anuragv/Box/Projects/PMBB_GWAS_PheWAS/MONDO/mondo_parent_child_relationship.txt", sep="\t")

#Remove the NAN parent_id

In [3]:
mondo_hie_not_null=mondo_hie[pd.notnull(mondo_hie["parent_id"])]

In [4]:
mondo_hie_not_null.head()

,parent_id,parent_desc,child_id,child_desc
0,MONDO_0021096,papillary epithelial neoplasm,MONDO_0021098,papillomatosis
1,MONDO_0021350,neoplasm of thorax,MONDO_0021100,breast neoplasm
2,MONDO_0015066,"neuroendocrine tumor of the appendix, well dif...",MONDO_0021101,appendix L-cell glucagon-like peptide-producin...
3,MONDO_0021259,prostate neoplasm,MONDO_0021102,prostate phyllodes tumor
4,MONDO_0004790,fatty liver disease,MONDO_0021104,alcoholic fatty liver disease


In [5]:
#ICD to mondo_mapping
mondo_icd_clean=pd.read_csv("/Users/anuragv/Box/Projects/PMBB_GWAS_PheWAS/MONDO/icd_mondo_mapping_clean_num_removed_icd_str.csv", sep=",")
mondo_icd_clean

,Unnamed: 0,ICD_TYPE,ICD_CODE,MONDO,MONDO_label,mapping_method,hops_from_MONDO_0000001
0,0,ICD10CM,A00.0,MONDO_0021678,gram-negative bacterial infections,mondo skos:exactMatch snomed -> shared cui;,4.0
1,1,ICD10CM,A00.9,MONDO_0021678,gram-negative bacterial infections,mondo skos:exactMatch snomed -> shared cui;,4.0
2,2,ICD10CM,A01.0,MONDO_0000827,salmonellosis,mondo skos:exactMatch snomed -> shared cui;,5.0
3,3,ICD10CM,A01.00,MONDO_0000827,salmonellosis,mondo skos:exactMatch snomed -> shared cui;,5.0
4,4,ICD10CM,A01.1,MONDO_0000827,salmonellosis,mondo skos:exactMatch snomed -> shared cui;,5.0
5,5,ICD10CM,A01.4,MONDO_0000827,salmonellosis,mondo skos:exactMatch snomed -> shared cui;,5.0
6,6,ICD10CM,A02.0,MONDO_0000827,salmonellosis,"mondo oboInOwl:hasDbXref icd10;""mondo oboInOwl...",5.0
7,7,ICD10CM,A02.1,MONDO_0000888,gastrointestinal mucositis,"mondo oboInOwl:hasDbXref cui;""mondo oboInOwl:h...",3.0
8,8,ICD10CM,A02.1,MONDO_0000916,intestinal infectious disease,"mondo oboInOwl:hasDbXref cui;""mondo oboInOwl:h...",3.0
9,9,ICD10CM,A02.1,MONDO_0005020,intestinal disease,"mondo oboInOwl:hasDbXref cui;""mondo oboInOwl:h...",3.0


In [6]:
#Concat mondo hierarchy data and icd code mapping
#mondo-mondo
mondo_hie_not_null_cols=mondo_hie_not_null[['parent_id','child_id']]
mondo_hie_not_null_cols.columns=['source','target']

#mondo-icd pair
mondo_icd_clean_cols = mondo_icd_clean[['ICD_CODE','MONDO']]     
mondo_icd_clean_cols.columns=['source','target']

mondo_hie_not_null_icd_clean=mondo_hie_not_null_cols.append(mondo_icd_clean_cols,ignore_index=True)

In [9]:
mondo_hie_not_null_icd_clean.head()

,source,target
0,MONDO_0021096,MONDO_0021098
1,MONDO_0021350,MONDO_0021100
2,MONDO_0015066,MONDO_0021101
3,MONDO_0021259,MONDO_0021102
4,MONDO_0004790,MONDO_0021104


In [10]:
G = nx.from_pandas_edgelist(mondo_hie_not_null_cols,'source','target')
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 21970
Number of edges: 21966
Average degree:   1.9996


In [12]:
def icd_path_length(df,g,s,t):                                                                                                                                               
    edges=df[(df['source']== s) | (df['source']==  t)].values.tolist()                          
    G.add_edges_from(edges)        
    try:                                                                              
        n=nx.shortest_path_length(g,s,t) 
    except nx.NetworkXNoPath:                                     
        n='No Path' 
    except nx.NodeNotFound:                                
        n='No node'               
    out=s,t,n      
    G.remove_edges_from(edges) 
    return out   

In [29]:
#need to prepare this file beforehand
#pairwise ICD-ICD (ONLY pulling top 100 rows)
data_icd = pd.read_csv('/Users/anuragv/Box/Projects/PMBB_GWAS_PheWAS/MONDO/icd_code_mondo_label_pairwise.txt', 
                       sep="|",nrows=100)
data_icd.columns=['code1','code2']

In [27]:
data_icd.head()

,code1,code2
0,100,100.00
1,100,100.81
2,100,100.90
3,100,101.00
4,100,102.00


In [28]:
for row in data_icd.itertuples(index=False): 
    out=icd_path_length(mondo_icd_clean_cols,G,row.code1,row.code2) 
    print(out)         
    #with open('icd_pair_distance_out_4.8.txt', 'a') as writer: 
        #writer.write(str(out) + "\n") 

(100, 100.0, 'No node')
(100, 100.81, 'No node')
(100, 100.9, 'No node')
(100, 101.0, 'No node')
(100, 102.0, 'No node')
(100, 102.1, 'No node')
(100, 102.2, 'No node')
(100, 102.3, 'No node')
(100, 102.4, 'No node')
(100, 102.5, 'No node')
(100, 102.6, 'No node')
(100, 102.8, 'No node')
(100, 102.9, 'No node')
(100, 103.0, 'No node')
(100, 103.1, 'No node')
(100, 103.2, 'No node')
(100, 103.3, 'No node')
(100, 104.8, 'No node')
(100, 104.9, 'No node')
(100, 110.0, 'No node')
(100, 110.1, 'No node')
(100, 110.2, 'No node')
(100, 110.3, 'No node')
(100, 110.4, 'No node')
(100, 110.5, 'No node')
(100, 110.6, 'No node')
(100, 110.8, 'No node')
(100, 110.9, 'No node')
(100, 111.0, 'No node')
(100, 111.1, 'No node')
(100, 111.2, 'No node')
(100, 111.8, 'No node')
(100, 111.9, 'No node')
(100, 112.0, 'No node')
(100, 112.1, 'No node')
(100, 112.2, 'No node')
(100, 112.3, 'No node')
(100, 112.4, 'No node')
(100, 112.5, 'No node')
(100, 112.8, 'No node')
(100, 112.81, 'No node')
(100, 112.82, 